<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="cognitiveclass.ai logo">
</center>


# Machine Learning Foundation

## Course 4, Part d: Dimensionality Reduction DEMO


## Introduction

We will be using customer data from a [Portuguese wholesale distributor](https://archive.ics.uci.edu/ml/datasets/Wholesale+customers) for clustering. This data file is called `Wholesale_Customers_Data`.

It contains the following features:

* Fresh: annual spending (m.u.) on fresh products
* Milk: annual spending (m.u.) on milk products
* Grocery: annual spending (m.u.) on grocery products
* Frozen: annual spending (m.u.) on frozen products
* Detergents_Paper: annual spending (m.u.) on detergents and paper products
* Delicatessen: annual spending (m.u.) on delicatessen products
* Channel: customer channel (1: hotel/restaurant/cafe or 2: retail)
* Region: customer region (1: Lisbon, 2: Porto, 3: Other)

In this data, the values for all spending are given in an arbitrary unit (m.u. = monetary unit).


In [ ]:
# Define a function to suppress warnings
def warn(*args, **kwargs):  # *args: variable number of positional arguments, **kwargs: variable number of keyword arguments
    pass  # Do nothing when warnings are triggered

# Import the warnings module to handle warning messages
import warnings
# Override the default warn function with our custom one that does nothing
warnings.warn = warn

# Import necessary libraries for data analysis and visualization
import seaborn as sns  # seaborn: statistical data visualization library
import pandas as pd    # pandas: data manipulation and analysis library
import numpy as np     # numpy: numerical computing library for arrays and mathematical operations

In [ ]:
# Import necessary libraries for data analysis and visualization
import os                     # os: operating system interface for file and directory operations
import pandas as pd           # pandas: data manipulation and analysis library
import numpy as np            # numpy: numerical computing library for arrays and mathematical operations
import seaborn as sns         # seaborn: statistical data visualization library
import matplotlib.pyplot as plt  # matplotlib.pyplot: plotting library for creating visualizations

## TASK 1

 

__Task 1 includes Part 1 to Part 4 which covers data acquistion, preprocessing and PCA__


## Part 1

In this section, we will:

* Import the data and check the data types.
* Drop the channel and region columns as they won't be used since we focus on numeric columns for this example.
* Convert the remaining columns to floats if necessary.
* Copy this version of the data (using the `copy` method) to a variable to preserve it. We will be using it later.


In [ ]:
# Load the wholesale customers dataset from a URL into a pandas DataFrame
data = pd.read_csv(  # pd.read_csv: function to read CSV file into DataFrame
    'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/Wholesale_Customers_Data.csv',  # URL path to the CSV file
    sep=','  # sep: delimiter to use (comma-separated values)
)

In [ ]:
# Display the dimensions of the DataFrame (number of rows and columns)
data.shape  # Returns tuple: (number of rows, number of columns)

In [ ]:
# Display the first 5 rows of the DataFrame to preview the data
data.head()  # head(): returns the first n rows (default n=5)

In [ ]:
# Remove the 'Channel' and 'Region' columns from the DataFrame
data = data.drop(  # drop(): removes specified labels from rows or columns
    ['Channel', 'Region'],  # List of column names to drop
    axis=1  # axis=1: indicates we are dropping columns (axis=0 would be rows)
)

In [ ]:
# Display the data types of each column in the DataFrame
data.dtypes  # dtypes: returns the data type of each column

In [ ]:
# Convert all columns to float64 data type for numerical operations
for col in data.columns:  # Iterate through each column name in the DataFrame
    data[col] = data[col].astype(np.float64)  # astype(): converts the column to specified data type (np.float64: 64-bit floating point)

Preserve the original data.


In [ ]:
# Create a copy of the data to preserve the original preprocessed version
data_orig = data.copy()  # copy(): creates a deep copy of the DataFrame (changes to data won't affect data_orig)

## Part 2

As with the previous lesson, we need to ensure the data is scaled and (relatively) normally distributed.

* Examine the correlation and skew.
* Perform any transformations and scale data using your favorite scaling method.
* View the pairwise correlation plots of the new data.


In [ ]:
# Calculate the correlation matrix to measure relationships between variables
corr_mat = data.corr()  # corr(): computes pairwise correlation of columns

# Remove the diagonal values (self-correlation = 1.0) for clearer analysis
for x in range(corr_mat.shape[0]):  # Iterate through each row index
    corr_mat.iloc[x, x] = 0.0  # iloc[row, col]: set diagonal element to 0.0
    
corr_mat  # Display the correlation matrix

As before, the two categories with their respective most strongly correlated variable.


In [ ]:
# Find the feature with the highest absolute correlation for each column
corr_mat.abs().idxmax()  # abs(): gets absolute values of correlations; idxmax(): returns index of maximum value in each column

Examine the skew values and log transform. Looks like all of them need it.


In [ ]:
# Calculate skewness values for each column and filter those needing log transformation
log_columns = data.skew().sort_values(ascending=False)  # skew(): computes skewness; sort_values(): sorts in descending order
log_columns = log_columns.loc[log_columns > 0.75]  # Filter columns with skewness > 0.75 (right-skewed distributions)

log_columns  # Display the highly skewed columns

In [ ]:
# Apply log transformation to reduce skewness in the data
for col in log_columns.index:  # Iterate through each column that needs transformation
    data[col] = np.log1p(data[col])  # np.log1p(): computes log(1 + x) to handle zero values safely

Scale the data again. Let's use `MinMaxScaler` this time just to mix things up.


In [ ]:
# Import MinMaxScaler to scale features to a range [0, 1]
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
mms = MinMaxScaler()  # MinMaxScaler(): scales each feature to a given range (default [0, 1])

# Scale each column independently to the range [0, 1]
for col in data.columns:  # Iterate through each column
    data[col] = mms.fit_transform(data[[col]]).squeeze()  # fit_transform(): fits scaler and transforms data; squeeze(): removes single-dimensional entries

### MinMax Scaling Formula

The MinMaxScaler transforms each feature by scaling it to a given range (default [0, 1]):

$$X_{scaled} = \frac{X - X_{min}}{X_{max} - X_{min}}$$

Where:
- $X$ is the original value
- $X_{min}$ is the minimum value in the feature
- $X_{max}$ is the maximum value in the feature
- $X_{scaled}$ is the scaled value in the range [0, 1]

This transformation preserves the shape of the original distribution while ensuring all features are on the same scale.

Visualize the relationship between the variables.


In [ ]:
# Configure and create a pairplot visualization to show relationships between all features
sns.set_context('notebook')  # set_context(): sets the plotting context to 'notebook' for appropriate sizing
sns.set_style('white')  # set_style(): sets the aesthetic style to 'white' (minimal background)
sns.pairplot(data);  # pairplot(): creates a grid of scatter plots for each pair of features with histograms on diagonal

## Part 3

In this section, we will:
* Using Scikit-learn's [pipeline function](http://scikit-learn.org/stable/modules/pipeline.html), recreate the data pre-processing scheme above (transformation and scaling) using a pipeline. If you used a non-Scikit learn function to transform the data (e.g. NumPy's log function), checkout  the custom transformer class called [`FunctionTransformer`](http://scikit-learn.org/stable/modules/preprocessing.html#custom-transformers).
* Use the pipeline to transform the original data that was stored at the end of question 1.
* Compare the results to the original data to verify that everything worked.

*Note:* Scikit-learn has a more flexible `Pipeline` function and a shortcut version called `make_pipeline`. Either can be used. Also, if different transformations need to be performed on the data, a [`FeatureUnion`](http://scikit-learn.org/stable/modules/pipeline.html#featureunion-composite-feature-spaces) can be used.


In [ ]:
# Import necessary classes for creating a preprocessing pipeline
from sklearn.preprocessing import FunctionTransformer  # For creating custom transformers from functions
from sklearn.pipeline import Pipeline  # For chaining multiple preprocessing steps

# Create a custom transformer that applies numpy's log1p function
log_transformer = FunctionTransformer(np.log1p)  # FunctionTransformer(): wraps np.log1p function for use in pipeline

# Define the pipeline steps as a list of (name, transformer) tuples
estimators = [
    ('log1p', log_transformer),  # Step 1: apply log transformation
    ('minmaxscale', MinMaxScaler())  # Step 2: apply MinMax scaling
]
# Create the pipeline by chaining the transformers
pipeline = Pipeline(estimators)  # Pipeline(): chains multiple transformers together

# Apply the entire pipeline to the original data
data_pipe = pipeline.fit_transform(data_orig)  # fit_transform(): fits the pipeline and transforms the data in one step

The results are identical. Note that machine learning models and grid searches can also be added to the pipeline (and in fact, usually are.)


In [ ]:
# Verify that the pipeline produces identical results to manual transformation
np.allclose(data_pipe, data)  # allclose(): returns True if two arrays are element-wise equal within a tolerance

## Part 4

In this section, we will:
* Perform PCA with `n_components` ranging from 1 to 5. 
* Store the amount of explained variance for each number of dimensions.
* Also store the feature importance for each number of dimensions. *Hint:* PCA doesn't explicitly provide this after a model is fit, but the `components_` properties can be used to determine something that approximates importance. How you decided to do so is entirely up to you.
* Plot the explained variance and feature importances.


In [ ]:
# Import PCA for dimensionality reduction
from sklearn.decomposition import PCA

# Initialize lists to store PCA models and feature importance weights
pca_list = list()
feature_weight_list = list()

# Fit PCA models with different numbers of components (1 to 5)
for n in range(1, 6):  # n: number of principal components to keep
    
    # Create and fit a PCA model with n components
    PCAmod = PCA(n_components=n)  # PCA(): creates a PCA model; n_components: number of components to keep
    PCAmod.fit(data)  # fit(): computes principal components from the data
    
    # Store the model along with total explained variance
    pca_list.append(pd.Series({
        'n': n,  # Number of components
        'model': PCAmod,  # Fitted PCA model
        'var': PCAmod.explained_variance_ratio_.sum()  # explained_variance_ratio_: proportion of variance explained by each component
    }))
    
    # Calculate feature importance by summing absolute component values
    abs_feature_values = np.abs(PCAmod.components_).sum(axis=0)  # components_: principal axes in feature space; sum(axis=0): sum across components
    feature_weight_list.append(pd.DataFrame({
        'n': n,  # Number of components
        'features': data.columns,  # Feature names
        'values': abs_feature_values / abs_feature_values.sum()  # Normalized feature importance
    }))
    
# Combine all PCA results into a DataFrame
pca_df = pd.concat(pca_list, axis=1).T.set_index('n')  # concat(): combines Series; T: transposes; set_index(): sets 'n' as index
pca_df

### Principal Component Analysis (PCA) Formulas

PCA transforms the data into a new coordinate system where the greatest variance lies on the first coordinate (first principal component), the second greatest variance on the second coordinate, and so on.

**1. Data Centering:**
$$X_{centered} = X - \mu$$

Where $\mu$ is the mean of each feature.

**2. Covariance Matrix:**
$$C = \frac{1}{n-1} X_{centered}^T X_{centered}$$

Where $n$ is the number of samples.

**3. Eigendecomposition:**
$$C v_i = \lambda_i v_i$$

Where:
- $v_i$ are the eigenvectors (principal components)
- $\lambda_i$ are the eigenvalues (variance explained by each component)

**4. Projection onto Principal Components:**
$$Z = X_{centered} \cdot V_k$$

Where $V_k$ contains the first $k$ eigenvectors.

**5. Explained Variance Ratio:**
$$\text{Explained Variance Ratio}_i = \frac{\lambda_i}{\sum_{j=1}^{p} \lambda_j}$$

This represents the proportion of total variance explained by the $i$-th principal component.

Create a table of feature importances for each data column.


In [ ]:
# Create a pivot table showing feature importance across different numbers of components
features_df = (
    pd.concat(feature_weight_list)  # Combine all feature weight DataFrames
    .pivot(  # pivot(): reshapes data (long to wide format)
        index='n',  # index: rows will be number of components
        columns='features',  # columns: columns will be feature names
        values='values'  # values: cell values will be feature importance weights
    )
)

features_df  # Display the feature importance table

Create a plot of explained variances.


In [ ]:
# Set larger text size for the plot and create a bar chart of explained variance
sns.set_context('talk')  # set_context('talk'): increases font sizes for presentations
ax = pca_df['var'].plot(kind='bar')  # plot(): creates a plot; kind='bar': specifies bar chart type

# Configure the plot labels and title
ax.set(
    xlabel='Number of dimensions',  # Label for x-axis
    ylabel='Percent explained variance',  # Label for y-axis
    title='Explained Variance vs Dimensions'  # Chart title
);

And here's a plot of feature importances.


In [ ]:
# Create a bar chart showing feature importance for each number of dimensions
ax = features_df.plot(
    kind='bar',  # kind='bar': creates a grouped bar chart
    figsize=(13, 8)  # figsize: sets figure size in inches (width, height)
)
ax.legend(loc='upper right')  # legend(): adds legend; loc: specifies location
# Configure the plot labels and title
ax.set(
    xlabel='Number of dimensions',  # Label for x-axis
    ylabel='Relative importance',  # Label for y-axis
    title='Feature importance vs Dimensions'  # Chart title
);

This concludes "Demo lab: Dimensionality Reduction (Part 1)". We will be going over the remaining parts in the next module.


## TASK 2

 

__Task 2 covers part 5 and 6. It includes KernelPCA and Model accuracy.__

 

>Note: Task 1 is a pre-requisite for this task. Please make sure you complete Task 1 before proceeding to Task 2.


## Part 5

In this section, we will:
* Fit a `KernelPCA` model with `kernel='rbf'`. You can choose how many components and what values to use for the other parameters (`rbf` refers to a Radial Basis Function kernel, and the `gamma` parameter governs scaling of this kernel and typically ranges between 0 and 1). Several other [kernels](https://scikit-learn.org/stable/modules/metrics.html) can be tried, and even passed ss cross validation parameters (see this [example](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html)).
* If you want to tinker some more, use `GridSearchCV` to tune the parameters of the `KernelPCA` model. 

The second step is tricky since grid searches are generally used for supervised machine learning methods and rely on scoring metrics, such as accuracy, to determine the best model. However, a custom scoring function can be written for `GridSearchCV`, where larger is better for the outcome of the scoring function. 

What would such a metric involve for PCA? What about percent of explained variance? Or perhaps the negative mean squared error on the data once it has been transformed and then inversely transformed?


In [ ]:
# Import necessary modules for Kernel PCA and hyperparameter tuning
from sklearn.decomposition import KernelPCA  # KernelPCA: non-linear dimensionality reduction using kernels
from sklearn.model_selection import GridSearchCV  # GridSearchCV: exhaustive search over parameter grid
from sklearn.metrics import mean_squared_error  # mean_squared_error: calculates MSE between actual and predicted values

# Define a custom scoring function for GridSearchCV (higher is better)
def scorer(pcamodel, X, y=None):  # pcamodel: PCA model to evaluate; X: input data; y: not used (None)

    try:
        X_val = X.values  # Try to extract numpy array from pandas DataFrame
    except:
        X_val = X  # If X is already a numpy array, use it directly
        
    # Transform data to lower dimensions and back to original space
    data_inv = pcamodel.fit(X_val).transform(X_val)  # transform(): projects data onto principal components
    data_inv = pcamodel.inverse_transform(data_inv)  # inverse_transform(): reconstructs original features from components
    
    # Calculate reconstruction error using mean squared error
    mse = mean_squared_error(data_inv.ravel(), X_val.ravel())  # ravel(): flattens arrays to 1D
    
    # Return negative MSE since GridSearchCV maximizes the score (lower MSE is better)
    return -1.0 * mse

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0],  # gamma: kernel coefficient for RBF (controls kernel width)
    'n_components': [2, 3, 4]  # n_components: number of components to keep
}

# Create a GridSearchCV object to find best KernelPCA parameters
kernelPCA = GridSearchCV(
    KernelPCA(kernel='rbf', fit_inverse_transform=True),  # kernel='rbf': Radial Basis Function kernel; fit_inverse_transform=True: enables reconstruction
    param_grid=param_grid,  # Parameter combinations to try
    scoring=scorer,  # Custom scoring function to optimize
    n_jobs=-1  # n_jobs=-1: use all available CPU cores for parallel processing
)

# Fit the grid search to find optimal parameters
kernelPCA = kernelPCA.fit(data)  # fit(): tries all parameter combinations and finds the best one

kernelPCA.best_estimator_  # Display the best KernelPCA model found

### Kernel PCA with RBF Kernel Formulas

Kernel PCA extends standard PCA to capture non-linear patterns by using the kernel trick.

**1. RBF (Radial Basis Function) Kernel:**
$$K(x, y) = \exp\left(-\gamma \|x - y\|^2\right)$$

Where:
- $x, y$ are data points
- $\gamma$ is the kernel coefficient (controls kernel width)
- $\|x - y\|^2$ is the squared Euclidean distance between points

**2. Kernel Matrix:**
$$K_{ij} = K(x_i, x_j)$$

The kernel matrix $K$ contains pairwise kernel values for all data points.

**3. Centered Kernel Matrix:**
$$\tilde{K} = K - 1_n K - K 1_n + 1_n K 1_n$$

Where $1_n$ is an $n \times n$ matrix with all elements equal to $\frac{1}{n}$.

**4. Eigendecomposition of Centered Kernel Matrix:**
$$\tilde{K} \alpha_i = \lambda_i \alpha_i$$

Where $\alpha_i$ are the eigenvectors and $\lambda_i$ are the eigenvalues.

**5. Reconstruction Error (MSE):**
$$MSE = \frac{1}{m} \sum_{i=1}^{m} (x_i - \hat{x}_i)^2$$

Where $x_i$ is the original data and $\hat{x}_i$ is the reconstructed data after transforming and inverse transforming.

## Part 6

Let's explore how our model accuracy may change if we include a `PCA` in our model building pipeline. Let's plan to use sklearn's `Pipeline` class and create a pipeline that has the following steps:
<ol>
  <li>A scaler</li>
  <li>`PCA(n_components=n)`</li>
  <li>`LogisticRegression`</li>
</ol>

* Load the Human Activity data from the datasets.
* Write a function that takes in a value of `n` and makes the above pipeline, then predicts the "Activity" column over a 5-fold StratifiedShuffleSplit, and returns the average test accuracy
* For various values of n, call the above function and store the average accuracies.
* Plot the average accuracy by number of dimensions.


In [ ]:
# Load the Human Activity Recognition dataset from a URL
data = pd.read_csv(  # pd.read_csv: function to read CSV file into DataFrame
    'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/Human_Activity_Recognition_Using_Smartphones_Data.csv',  # URL path to the dataset
    sep=','  # sep: delimiter to use (comma-separated values)
)

In [ ]:
# Display all column names in the dataset
data.columns  # columns: returns the column labels of the DataFrame

In [ ]:
# Import necessary modules for building and evaluating a machine learning pipeline
from sklearn.pipeline import Pipeline  # Pipeline: chains preprocessing and model steps
from sklearn.preprocessing import StandardScaler  # StandardScaler: standardizes features by removing mean and scaling to unit variance
from sklearn.model_selection import StratifiedShuffleSplit  # StratifiedShuffleSplit: creates stratified train/test splits
from sklearn.linear_model import LogisticRegression  # LogisticRegression: classification model
from sklearn.metrics import accuracy_score  # accuracy_score: calculates classification accuracy

# Separate features (X) and target variable (y)
X = data.drop('Activity', axis=1)  # Drop the target column; axis=1: drop column
y = data.Activity  # Extract the Activity column as target

# Create a stratified cross-validation splitter
sss = StratifiedShuffleSplit(n_splits=5, random_state=42)  # n_splits=5: create 5 train/test splits; random_state=42: ensures reproducibility

# Define function to evaluate average accuracy for a given number of PCA components
def get_avg_score(n):  # n: number of PCA components to use
    # Define the pipeline steps
    pipe = [
        ('scaler', StandardScaler()),  # Step 1: standardize features (zero mean, unit variance)
        ('pca', PCA(n_components=n)),  # Step 2: reduce dimensionality to n components
        ('estimator', LogisticRegression(solver='liblinear'))  # Step 3: train logistic regression; solver='liblinear': optimization algorithm
    ]
    pipe = Pipeline(pipe)  # Create the pipeline
    scores = []  # List to store accuracy scores for each fold
    
    # Iterate through each train/test split
    for train_index, test_index in sss.split(X, y):  # split(): generates train/test indices
        X_train, X_test = X.loc[train_index], X.loc[test_index]  # Split features
        y_train, y_test = y.loc[train_index], y.loc[test_index]  # Split target
        pipe.fit(X_train, y_train)  # fit(): train the entire pipeline on training data
        scores.append(accuracy_score(y_test, pipe.predict(X_test)))  # predict(): make predictions; accuracy_score(): calculate accuracy
    
    return np.mean(scores)  # Return average accuracy across all folds

# Test different numbers of PCA components
ns = [10, 20, 50, 100, 150, 200, 300, 400]  # List of component numbers to try
score_list = [get_avg_score(n) for n in ns]  # Calculate average accuracy for each n

### Machine Learning Pipeline Formulas

This pipeline combines StandardScaler, PCA, and Logistic Regression.

**1. Standard Scaling:**
$$z = \frac{x - \mu}{\sigma}$$

Where:
- $x$ is the original feature value
- $\mu$ is the mean of the feature
- $\sigma$ is the standard deviation of the feature
- $z$ is the standardized value

**2. Logistic Regression:**

The logistic regression model predicts probabilities using the sigmoid function:

$$P(y=1|x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n)}}$$

Or in matrix form:
$$P(y=1|x) = \frac{1}{1 + e^{-\beta^T x}}$$

**3. Cross-Entropy Loss Function:**
$$J(\beta) = -\frac{1}{m} \sum_{i=1}^{m} \left[y_i \log(h_\beta(x_i)) + (1-y_i) \log(1-h_\beta(x_i))\right]$$

Where:
- $m$ is the number of training examples
- $y_i$ is the actual label (0 or 1)
- $h_\beta(x_i)$ is the predicted probability

**4. Accuracy Score:**
$$\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Total number of predictions}}$$

In [ ]:
# Set larger text size for the plot
sns.set_context('talk')  # set_context('talk'): increases font sizes for presentations

# Create a matplotlib axes object for the plot
ax = plt.axes()  # axes(): creates a new axes object for plotting
# Plot accuracy scores vs number of dimensions
ax.plot(ns, score_list)  # plot(): creates a line plot; ns: x-values (number of dimensions); score_list: y-values (accuracy scores)

# Configure the plot with labels and title
ax.set(
    xlabel='Number of Dimensions',  # Label for x-axis
    ylabel='Average Accuracy',  # Label for y-axis
    title='LogisticRegression Accuracy vs Number of dimensions on the Human Activity Dataset'  # Chart title
)
ax.grid(True)  # grid(): adds a grid to the plot; True: enables grid lines

---
### Machine Learning Foundation (C) 2020 IBM Corporation
